The goal of this notebook is to write python code to aproximate the price of an ATMF straddle. An ATMF straddle (or At-The-Money Forward straddle) is a type of investment strategy used primarily in options trading. 

An option is considered "ATM" (or At-The-Money) when the strike price of the option is equal to the current price of the underlying asset. A straddle involves buying a call option and a put option with the same strike price/same expiration date. The idea behind a straddle is to profit from significant price movements on the underlying asset regardless of direction. The Forward element of this ATMF straddle dictates that the strike price is set at the forward price expected at the expiration of the option. The forward price is typically calculated based on the current prices plus any carry costs (mainly interests or dividends) until the option's maturity.

From JPMorgan Python training course, here is the formula to approximate the price of an ATMF straddle:


$$STRADDLE_ATMF=(2/sqrt(2*pi))*F*rho*sqrt(T)$$

with:

$$rho=implied volatily$$
$$T=time to maturity$$
$$F=forward of the underlier$$
$$pi=pi (of course)$$

Components:
- 1/sqrt(2*pi): This component is a part of the normalization factor used in the probability density function of a standard normal distribution, often used in formulas derived from the Black-Scholes model.
- vol(rho): Implied volatility represents the expected fluctuation in the underlying asset's price. Higher volatility increases the premium of both call and put options because the expected movement of the stock price away from the strike price increases.
- sqrt(time): The square root of time reflects how option prices are sensitive not just to the length of time to expiration but to the square root of this time, which aligns with the principles of stochastic processes where variance of stock price returns increases with time.

This formula effectively calculates the price of an ATMF straddle by estimating the expected magnitude of the underlying asset's price movement whithin the given time frame, under the assumption of a log-normal distribution of price changes (a common assumption in financial models).
- 2x: This represents buying both a call and a put. Since we are buying both a call and a put where both options are at-the-money, their premiums can be expected to be similar, hence the factor of 2.
- Volatility and Time: The product of volatility and the square root of time gives a measure of the expected range (standard deviation) of the price movement. Multiplying this by the normalization constant of the normal distribution adjusts this range into a probability-weighted value.

Lets start by defining these variables and create the formula and calling it.



In [1]:
vol=0.2 #this is rho, implied volatiliy
time=1. #time to maturity, adding the "." to make it a float instead of int

#define a function called straddlePricer to be called whenever necessary. Note, **=sqrt
def straddlePricer(vol,time):
    return 2. * ((1./(2*3.14) ** 0.5) * vol * time ** 0.5)

print(straddlePricer(vol,time))

0.15961737689352445


This result can be interpreted as the cost of buying both the call and the put option at-the-money forward, as a fraction of the underlying asset's price. In other words, it's bassically 15.93% of the current price of the underlying asset. Let's try making the inputs optional.

In [2]:
def straddlePricer(vol=0.2,time=1.0):
    return 2. * ((1./(2*3.14) ** 0.5) * vol * time ** 0.5)

print(straddlePricer())
print(straddlePricer(0.22))
print(straddlePricer(0.22, 2.0))

0.15961737689352445
0.1755791145828769
0.24830636511256418


Next step, lets try getting rid of the approximation of pi and the **0.5 for sqrt in my function using numpy as np.

In [3]:
import numpy as np
def straddlePricer(vol=0.2,time=1.0):
    return 2. * ((1./np.sqrt(2*np.pi)) * vol * np.sqrt(time))

print(straddlePricer())


0.1595769121605731


Next step, lets try pricing the straddle using the Monte Carlo method. 

The Monte Carlo method is a broad class of computational algorithms that rely on repeated random sampling to obtain numerical results. 

I won't go into more details of Monte Carlo simulations (see my Physics/Engineering repositories to see where I've learned and applied about Monte Carlo). Monte Carlo is very popular in finance such as risk assessment and modeling financial instruments like options and derivatives.

Lets generate a normally distributed set of random numbers to simulate the asset's movement through time.

In [4]:
def straddlePricerMC(vol=0.2,time=1.0, mcPaths=100):
    dailyVol=vol/np.sqrt(252.) #252 for 252 trading days in a year since we are looking for daily volatility
    resultSum=0
    for p in range(mcPaths):
        resultSum+=np.abs(np.prod((1+np.random.normal(0, dailyVol, int(round(time*252)))))-1)
    return resultSum/mcPaths

#for fun, lets compare the result of both functions
print(straddlePricer())
print(straddlePricerMC())

0.1595769121605731
0.1529257084696273


Lets talk a bit about what is going on here.

We know variance scales linearly with time. This leaves us with two options:

1. divide the variance by time and take the square root to get a daily volatility, or
2. take the square root of variance (volatility) and divide by the root of time

Lets compare the two methods.

In [6]:
#comparing the two methods
vol=0.2
var=vol**2
sqrtVarOverTime=np.sqrt(var/252)
volOverSqrtTime=vol/np.sqrt(252)
valuesEqual=np.isclose(sqrtVarOverTime, volOverSqrtTime)
print(f'sqrtVarOverTime={sqrtVarOverTime}\nvolOverSqrtTime={volOverSqrtTime}\nAre they close? {valuesEqual}')

sqrtVarOverTime=0.012598815766974242
volOverSqrtTime=0.01259881576697424
Are they close? True


Lets set the default value of our comulative sum to be 0. Next we have a loop using range(mcPaths) to return an iterator over a list of ints starting at 0 and going to n-1.

In [14]:
resultSum=0
range10=range(10)
lst=list(range10)
print(lst)
print(len(lst))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
10


Lets use 

In [15]:
def straddlePricerMC(vol=0.2, time=1.0, mcPaths=100):
    dailyVol=vol/np.sqrt(252.)
    resultSum=0
    for _ in range (mcPaths):   #setting "p" at "_" since we don't want to do anything with p
        resultSum+=np.abs(np.prod(1+(np.random.normal(0, dailyVol, int(round(time*252)))))-1)
    return resultSum/mcPaths

In [16]:
straddlePricerMC()

0.16093237844012506

Now, given we have an asset return timeseries, how much is a straddle woth? We're interest in the terminal value of the asset and because we assume the straddle is struck ATM, we can just take the absolute value of the asset's deviation from the initial value (in this case, 1)

In [17]:
time=1
nDays=time*252
dailyVol=vol/np.sqrt(252.)
print(nDays)

np.random.normal(0,dailyVol, nDays)

252


array([ 5.34134400e-03, -1.02877781e-04,  1.03994253e-03,  1.03030066e-02,
        3.31071770e-03,  6.49871147e-03, -1.57612996e-03, -8.13647748e-03,
       -2.01130758e-02,  2.15709792e-02, -1.06799184e-02, -2.33028779e-03,
       -1.99369217e-02,  7.12142111e-03,  2.48361333e-02, -1.73204936e-02,
        1.12970615e-02, -4.11589442e-04, -2.23902091e-02, -7.02857696e-03,
       -1.68632071e-02,  4.24127744e-03,  1.28802493e-03,  7.24887233e-04,
       -8.06709417e-03,  2.62045155e-02, -5.40299964e-03, -2.42084076e-02,
        2.57618917e-03, -2.85790852e-02, -9.02400255e-03, -2.55737873e-02,
       -1.58197193e-02, -2.83620435e-02,  3.25116718e-03, -6.02998242e-03,
        1.51151507e-03, -1.28152147e-02, -1.08670636e-03, -4.42761314e-03,
        8.07716925e-03, -6.32018268e-03, -6.89819590e-03, -1.48284682e-03,
       -1.27379285e-03,  2.08981055e-03,  2.89444506e-03,  7.03591689e-03,
       -1.23581396e-02,  3.76177567e-02,  9.60764294e-03, -1.07134075e-02,
        7.45645771e-03,  

In [18]:
np.random.seed(42) #garantee the same result from the two random series

returns=np.random.normal(0,dailyVol, time*252)
priceAtMaturity=np.prod(1+returns)
changeAtMaturity=priceAtMaturity-1
absChangeAtMaturity=np.abs(changeAtMaturity)
print(absChangeAtMaturity)

0.030088573823511933


Lets take another look at what we did using  pandas and perspective

In [19]:
import pandas as pd
from perspective import psp
print(pd.__version__)

ModuleNotFoundError: No module named 'pandas'

In [20]:
mcPaths=100
resultSum=0.
for _ in range(mcPaths):
    resultSum+=np.abs(np.prod(1+np.random.normal(0., dailyVol, time*252))-1)
print(resultSum/mcPaths)

0.14729794955899883


Lets use more paths to converge to our original price. We can also check the time our function takes to run.

In [22]:
straddlePricerMC(mcPaths=2000)
%timeit straddlePricerMC(mcPaths=2000)

29.8 ms ± 178 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


Lets do multiple different paths

In [23]:
print(f"1 path: {straddlePricerMC(mcPaths=1)}")
print(f"2000 path: {straddlePricerMC(mcPaths=2000)}")
print(f"5000 path: {straddlePricerMC(mcPaths=5000)}")
print(f"10000 path: {straddlePricerMC(mcPaths=10000)}")
print(f"100000 path: {straddlePricerMC(mcPaths=100000)}")
print(f"Closed form approximation {straddlePricer()}")

1 path: 0.021322617958779544
2000 path: 0.16044298937916354
5000 path: 0.16131340073073527
10000 path: 0.15926525612955403
100000 path: 0.15950484412394839
Closed form approximation 0.1595769121605731
